In [4]:
from pathlib import Path
import pandas as pd
import yaml
from tqdm import tqdm
import cv2
import shutil
import numpy as np
np.random.seed(42)

In [5]:
source_dataset_dir = Path('./dataset')
classes_path = source_dataset_dir / 'classes.txt'
images_dir = Path('data_new/images')

labels_dir = Path('data_new/labels')
(images_dir / "train").mkdir(exist_ok=True, parents=True)
(images_dir / "val").mkdir(exist_ok=True, parents=True)
(labels_dir / "train").mkdir(exist_ok=True, parents=True)
(labels_dir / "val").mkdir(exist_ok=True, parents=True)


In [6]:
def convert_center_to_corners(x_center, y_center, width, height):
    x_bottom_right = x_center + width / 2
    y_bottom_right = y_center + height / 2

    x_upper_left = x_center - width / 2
    y_upper_left = y_center - height / 2

    return (x_bottom_right, y_bottom_right), (x_upper_left, y_upper_left)


In [20]:
def get_coco_cord(text_path: Path):
    annotation = open(text_path).read()
    new_annotation = []
    for i, line in enumerate(annotation.splitlines()):
        class_id, *points = line.split()
        points =  list(map(float, points))
        x_c, y_c = points[0], points[1]
        w, h = points[2], points[3]
        x = (x_c - w / 2) 
        y = (y_c - h / 2) 
        X = (x_c + w / 2) 
        Y = (y_c + h / 2)
        points = [x, y, x, Y, X, Y, X, y]
        points =  list(map(str, points))
        new_line = class_id + " " + " ".join(points)
        new_annotation.append(new_line)
    return new_annotation
        

In [21]:
for txt_file in tqdm(source_dataset_dir.glob('*.txt')):
    if "classes" in txt_file.stem:
        continue
    image_file = txt_file.with_suffix(".jpg")
    subfolder = Path("train") if np.random.rand() < 0.9 else Path("val")
    
    #shutil.copy(txt_file, labels_dir / subfolder / txt_file.name)
    with open(labels_dir / subfolder / txt_file.name, 'wt') as fil:
        fil.write("\n".join(get_coco_cord(txt_file)))
    shutil.copy(image_file, images_dir / subfolder / image_file.name)

1163it [00:03, 318.39it/s]


In [22]:
import yaml
result_yaml = {
    "path": "/home/simba9/pet/shov/data_new",
    "train": "images/train",
    "val": "images/val"
}
with open(classes_path) as f:
    names = []
    for line in f:
        names.append(line.strip())
    
    result_yaml["names"] = dict(zip(list(range(len(names))), names))
with open(Path('./shov.yml'), 'w') as yaml_file:
    yaml.dump(result_yaml, yaml_file, default_flow_style=False)
